In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

2.6.0+cu124


In [ ]:
!pip install -q torch_geometric
!pip install -q class_resolver
!pip3 install pymatting


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 2.4 MB/s eta 0:00:00


In [1]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, log_loss
from scipy import sparse
from scipy.sparse.linalg import eigsh

In [2]:
fa_feature_path = "/home/snu/Downloads/Histogram_CN_FA_20bin_updated.npy"
Histogram_feature_CN_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Load AD features
fa_feature_path = "/home/snu/Downloads/Histogram_AD_FA_20bin_updated.npy"
Histogram_feature_AD_FA_array = np.load(fa_feature_path, allow_pickle=True)

# Combine features and labels
X = np.vstack([Histogram_feature_CN_FA_array, Histogram_feature_AD_FA_array])
y = np.hstack([
    np.zeros(Histogram_feature_CN_FA_array.shape[0], dtype=np.int64),
    np.ones(Histogram_feature_AD_FA_array.shape[0], dtype=np.int64)
])
np.random.seed(42)
perm = np.random.permutation(X.shape[0])
X = X[perm]
y = y[perm]
num_nodes, num_feats = X.shape
print(f"Features: {X.shape}, Labels: {y.shape}")

Features: (223, 180), Labels: (223,)


In [3]:
# F = np.concatenate((Histogram_feature_CN_FA_array, Histogram_feature_AD_FA_array), axis=0)
# F = F.astype(np.float32)
# print("Combined Shape:", F.shape)
F = X
F = F.astype(np.float32)
print("Final Shape:", F.shape)
print("Data Type:", F.dtype)

Final Shape: (223, 180)
Data Type: float32


In [4]:
def tokencut_on_features(F_array, alpha=1e-6):
    """
    Apply TokenCut clustering to feature matrix F_array (shape: N × D).
    Returns binary labels (0/1) for each node.
    """
    N, D = F_array.shape

    # 1. Normalize features row-wise
    norms = np.linalg.norm(F_array, axis=1, keepdims=True) + 1e-10
    F_norm = F_array / norms

    # 2. Construct cosine similarity matrix (fully connected)
    W = np.dot(F_norm, F_norm.T)
    W = W + alpha  # stabilizer

    # 3. Normalized Laplacian: L = I - D^{-1/2} W D^{-1/2}  where, W is the similarity matrix and D is the degree matrix
    d = np.sum(W, axis=1)
    d_inv_sqrt = np.diag(1.0 / np.sqrt(d + 1e-10))
    L = np.eye(N) - d_inv_sqrt @ W @ d_inv_sqrt

    # Sparse for efficiency
    L_sparse = sparse.csr_matrix(L)

    # 4. Compute the Fiedler vector, i.e., the second smallest eigenvector of L.
    vals, vecs = eigsh(L_sparse, k=2, which='SM')
    fiedler = vecs[:, 1]

    # 5. Threshold by mean
    threshold = fiedler.mean()
    labels = (fiedler > threshold).astype(np.int64)

    return labels, fiedler

In [5]:
labels, scores = tokencut_on_features(X)

# Evaluate
y_pred = labels
acc = accuracy_score(y, y_pred)
inv_acc = accuracy_score(y, 1 - y_pred)
if inv_acc > acc:
    y_pred = 1 - y_pred
    acc = inv_acc

prec = precision_score(y, y_pred)
rec = recall_score(y, y_pred)
f1 = f1_score(y, y_pred)

# Normalize fiedler vector for log_loss
probs = (scores - scores.min()) / (scores.max() - scores.min() + 1e-10)
logloss = log_loss(y, probs)

print("===== TokenCut Results =====")
print("Accuracy Score:", acc)
print("Precision Score:", prec)
print("Recall Score:", rec)
print("F1 Score:", f1)
print("Log Loss:", logloss)

===== TokenCut Results =====
Accuracy Score: 0.7533632286995515
Precision Score: 0.6767676767676768
Recall Score: 0.7444444444444445
F1 Score: 0.708994708994709
Log Loss: 1.4180188913965375


In [6]:
print(probs)

[0.76534247 0.74587198 0.69619682 0.27572547 0.67508232 0.42557534
 0.65336252 0.59975612 0.5504863  0.42648936 0.91856585 0.69444596
 0.44815739 0.72853667 0.74594832 0.48246727 0.76306895 0.70181072
 0.57395068 0.8781697  0.56619059 0.70644876 0.77863331 0.35406706
 0.52506407 0.25530748 0.         0.65528577 0.79084465 0.8474146
 0.76880492 0.72201442 0.92449318 0.38119141 0.13702464 0.93436113
 0.59048379 0.67917868 0.71059131 0.19778996 0.28738664 0.67557216
 0.18542164 0.58141422 0.45086288 0.77889532 0.73056417 0.80176279
 0.38134409 0.34326913 0.61506776 0.49298254 0.3434946  0.59245175
 0.91422254 0.69967214 0.57322624 0.68171429 0.26807924 0.79945863
 0.37264001 0.83144054 0.78666444 0.66667396 0.78396789 0.41568439
 0.35297758 0.74840397 0.53528989 0.56545832 0.84064384 0.65122346
 0.7587819  0.57430411 0.57416922 0.87435305 0.71244114 0.86367891
 0.77790142 0.42554767 0.61284359 0.2614513  0.37168846 0.84117326
 0.44522093 0.26024497 0.6220778  0.71246054 0.48772974 0.66079

In [7]:
num_runs = 10
acc_scores, prec_scores, rec_scores, f1_scores, log_losses = [], [], [], [], []

for run in range(num_runs):
    print(f"\n--- Run {run+1}/{num_runs} ---")
    np.random.seed(run)
    perm = np.random.permutation(X.shape[0])
    X_run = X[perm]
    y_run = y[perm]

    labels, scores = tokencut_on_features(X_run)

    # Align labels to ground truth
    y_pred = labels
    acc = accuracy_score(y_run, y_pred)
    inv_acc = accuracy_score(y_run, 1 - y_pred)
    if inv_acc > acc:
        y_pred = 1 - y_pred
        acc = inv_acc

    prec = precision_score(y_run, y_pred)
    rec = recall_score(y_run, y_pred)
    f1 = f1_score(y_run, y_pred)
    probs = (scores - scores.min()) / (scores.max() - scores.min() + 1e-10)
    logloss = log_loss(y_run, probs)

    acc_scores.append(acc)
    prec_scores.append(prec)
    rec_scores.append(rec)
    f1_scores.append(f1)
    log_losses.append(logloss)

    print(f"Run {run+1} | Acc: {acc:.4f} | Prec: {prec:.4f} | Rec: {rec:.4f} | F1: {f1:.4f} | LogLoss: {logloss:.4f}")

print("\n================ FINAL SUMMARY ================\n")
print(f"{'Metric':>15} | {'Mean':>10} ± {'Std':<10}")
print("-" * 50)
print(f"{'Accuracy':>15} | {np.mean(acc_scores):.4f} ± {np.std(acc_scores):.4f}")
print(f"{'Precision':>15} | {np.mean(prec_scores):.4f} ± {np.std(prec_scores):.4f}")
print(f"{'Recall':>15} | {np.mean(rec_scores):.4f} ± {np.std(rec_scores):.4f}")
print(f"{'F1 Score':>15} | {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
print(f"{'Log Loss':>15} | {np.mean(log_losses):.4f} ± {np.std(log_losses):.4f}")


--- Run 1/10 ---
Run 1 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 0.5095

--- Run 2/10 ---
Run 2 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 1.4180

--- Run 3/10 ---
Run 3 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 1.4180

--- Run 4/10 ---
Run 4 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 1.4180

--- Run 5/10 ---
Run 5 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 1.4180

--- Run 6/10 ---
Run 6 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 0.5095

--- Run 7/10 ---
Run 7 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 1.4180

--- Run 8/10 ---
Run 8 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 1.4180

--- Run 9/10 ---
Run 9 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 0.5095

--- Run 10/10 ---
Run 10 | Acc: 0.7534 | Prec: 0.6768 | Rec: 0.7444 | F1: 0.7090 | LogLoss: 0.5095

================ 